In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [48]:
def generate_time_sequence(T=20, L=1000, N=100):
    np.random.seed(2)
    x = np.empty((N, L), 'int64')
    x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
    data = np.sin(x / 1.0 / T).astype('float64')
    return data

In [47]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)

        for input_t in input.split(1, dim=1):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.cat(outputs, dim=1)
        return outputs

In [8]:
epochs = 10
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)
# load data and make training set
data = generate_time_sequence()
input = torch.from_numpy(data[3:, :-1])
target = torch.from_numpy(data[3:, 1:])
test_input = torch.from_numpy(data[:3, :-1])
test_target = torch.from_numpy(data[:3, 1:])
# build the model
seq = Sequence()
seq.double()
criterion = nn.MSELoss()
# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
#begin to train
for i in range(epochs):
    print('STEP: ', i)
    def closure():
        optimizer.zero_grad()
        out = seq(input)
        loss = criterion(out, target)
        print('loss:', loss.item())
        loss.backward()
        return loss
    optimizer.step(closure)
    # begin to predict, no need to track gradient here
    with torch.no_grad():
        future = 1000
        pred = seq(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print('test loss:', loss.item())
        y = pred.detach().numpy()
    # draw the result
    plt.figure(figsize=(30,10))
    plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
    plt.xlabel('x', fontsize=20)
    plt.ylabel('y', fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    def draw(yi, color):
        plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
        plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
    draw(y[0], 'r')
    draw(y[1], 'g')
    draw(y[2], 'b')
    plt.savefig('predict%d.pdf'%i)
    plt.close()

STEP:  0
loss: 0.5023738122475573
loss: 0.49856639379435636
loss: 0.479011960611529
loss: 0.446334902148423
loss: 0.35406310257492973
loss: 0.20507016617681417
loss: 1.396053156116744
loss: 0.032494411484718044
loss: 0.029934875839604254
loss: 0.02832682101153418
loss: 0.026830612218823123
loss: 0.023771201989989676
loss: 0.018901413504545626
loss: 0.010646818233205624
loss: 0.008725752090268482
loss: 0.007872181287777214
loss: 0.005477842749594302
loss: 0.004051933564063765
loss: 0.002729622701159074
loss: 0.001540265276980803
test loss: 0.0013000876156955708
STEP:  1
loss: 0.0012797646167826966
loss: 0.001169055495471728
loss: 0.0011498916013727959
loss: 0.0011288245291281235
loss: 0.001063056134102498
loss: 0.0009564006443282015
loss: 0.0008210827589982917
loss: 0.0007670796029146066
loss: 0.0007294716423578395
loss: 0.0007246558169811719
loss: 0.0007206211393649049
loss: 0.0007126686053107041
loss: 0.0006961310498770009
loss: 0.0006641101811542689
loss: 0.0006101210702277351
loss: 